In [11]:
import streamlit as st
import pandas as pd
from openai import OpenAI
import os
import json
import time
from datetime import datetime
from dotenv import load_dotenv
from PIL import Image

In [20]:
load_dotenv('.env')
api_key=os.environ.get('OPENAI_KEY')

In [26]:
client = OpenAI(api_key=os.environ.get("OPENAI_KEY"))

In [24]:
# Streamlit App UI
st.title("AI-Powered Data Visualization App")
st.write("Upload a CSV or Excel file, enter your query, and let AI create the visualization for you.")

2025-01-09 23:10:16.560 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:16.721 
  command:

    streamlit run /Users/feekah/Library/Python/3.12/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-01-09 23:10:16.721 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:16.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:16.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:16.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:16.722 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [25]:
# File upload
uploaded_file = st.file_uploader("Choose a CSV or Excel file", type=["csv", "xlsx"])

# User query input
user_query = st.text_input("Enter your query for visualization (e.g., 'Visualize revenue by year')")

2025-01-09 23:10:39.396 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:39.398 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:39.400 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:39.402 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:39.403 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:39.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:39.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-09 23:10:39.405 Session state does not function when running a script without `streamlit run`
2025-01-09 23:10

In [27]:
# Process the uploaded file
if uploaded_file:
    # Load data into a DataFrame
    if uploaded_file.name.endswith('.csv'):
        df = pd.read_csv(uploaded_file)
    else:
        df = pd.read_excel(uploaded_file)
    
    st.write("### Uploaded Data Preview:")
    st.dataframe(df.head())

    unique_json_path = f"fin_data_{datetime.now().strftime('%Y%m%d%H%M%S')}.json"
    json_data = df.to_json(orient="columns")

    with open(unique_json_path, "w") as json_file:
        json.dump(json_data, json_file)

    # Upload file to OpenAI
    file = client.files.create(
        file=open(unique_json_path, "rb"),
        purpose='assistants'
    )

    # Define assistant instructions
    assistant = client.beta.assistants.create(
        instructions=f"You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization based on the query.",
        model="gpt-4o",
        tools=[{"type": "code_interpreter"}],
        tool_resources={"code_interpreter": {"file_ids": [file.id]}}
    )

    # Create a thread and send the user query
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": user_query
                    }
        ]
    )
    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    st.write("Generating plot...")

    # Check for completion and retrieve the plot
    while True:
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        try:
            # Check if image file is in response
            plot_file_id = messages.data[0].content[0].image_file.file_id
            time.sleep(5)  # Wait a bit to ensure run is completed
            break
        except:
            time.sleep(5)
            st.write("Assistant is still processing...")

    # Retrieve and display the plot
    def convert_file_to_png(file_id, write_path):
        data = client.files.content(file_id)
        data_bytes = data.read()
        with open(write_path, "wb") as file:
            file.write(data_bytes)

    # Convert plot to PNG and display
    image_path = "generated_plot.png"
    convert_file_to_png(plot_file_id, image_path)
    
    st.write("### Generated Plot:")
    st.image(image_path)

    # Clean up temporary files
    os.remove(unique_json_path)
    os.remove(image_path)
